In [1]:
import numpy as np
import os
from tqdm import tqdm, trange
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
from collections import defaultdict
from load_data import load_data,get_SG_FR, similarity_metric, read_Decoding_csv, load_neural_npz
from dataset import LinearRegressionModel, NeuralDataset
from oasis.functions import deconvolve

/home/friiz/anaconda3/envs/pbl/lib/python3.11/site-packages/oasis/functions.py:13: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  warn("Could not find cvxpy. Don't worry, you can still use OASIS, " +


In [2]:
# load data
Target_Brain_region = 'VISp'
data_root = '../data'
sessions = ['A','B']
session_datas = []
session_folders = [] 
for session in sessions:
    tgt_dict, tgt_folder = read_Decoding_csv(data_path='../data',region_type = Target_Brain_region, session_letter = session)
    session_datas.append(tgt_dict)
    session_folders.append(tgt_folder)
print('================================================')
print(session_folders)

ts_sessions = []
dff_sessions = []
all_roi_masks_sessions = []
cids_sessions = []
metadata_sessions = []
spike_info_sessions = []

for tgt_data_path in session_folders:
    for file_name in os.listdir(os.path.join(data_root, tgt_data_path)):
        if file_name.endswith('.npz'):
            ts, dff, all_roi_masks, cids, metadata = load_neural_npz(os.path.join(data_root, tgt_data_path,file_name))
            ts_sessions.append(ts)
            dff_sessions.append(dff)
            all_roi_masks_sessions.append(all_roi_masks)
            cids_sessions.append(cids)
            metadata_sessions.append(metadata)
            spike_info_sessions.append(np.array([deconvolve(dff[_i,:], penalty=1)[1] for _i in trange(dff.shape[0])]))
            position = metadata.tolist()
            print('================================================')
            print('Caution, the position of this file is about the brain part of', position['targeted_structure'])
            print('================================================')
print('================================================')

we have keys in data_dict, including:
stimulus_metadata_drifting_gratings.csv
stimulus_metadata_natural_movie_one.csv
stimulus_metadata_natural_movie_three.csv
stimulus_metadata_spontaneous.csv
we have keys in data_dict, including:
stimulus_metadata_natural_movie_one.csv
stimulus_metadata_natural_scenes.csv
stimulus_metadata_spontaneous.csv
stimulus_metadata_static_gratings.csv
stimulus_metadata_total.csv
['VISp_three_session_A_501704220', 'VISp_three_session_B_501559087']
check subtitles, should be ['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']


100%|██████████| 142/142 [00:37<00:00,  3.82it/s]


Caution, the position of this file is about the brain part of VISp
check subtitles, should be ['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']


100%|██████████| 174/174 [00:48<00:00,  3.61it/s]


Caution, the position of this file is about the brain part of VISp


In [3]:
print(session_datas)

[{'stimulus_metadata_drifting_gratings.csv':      Unnamed: 0  temporal_frequency  orientation  blank_sweep   start     end
0             0                 8.0        270.0          0.0     747     807
1             1                 2.0        135.0          0.0     837     897
2             2                 2.0        315.0          0.0     927     987
3             3                15.0        315.0          0.0    1018    1077
4             4                 1.0        270.0          0.0    1108    1168
..          ...                 ...          ...          ...     ...     ...
623         623                 4.0        270.0          0.0  114837  114897
624         624                 4.0        180.0          0.0  114928  114987
625         625                 2.0        180.0          0.0  115018  115078
626         626                 8.0        270.0          0.0  115108  115168
627         627                 4.0          0.0          0.0  115198  115258

[628 rows x 6 colu

In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import trange

# 定义目标脑区
target_brain_regions = ['VISp']#, 'VISal', 'VISl']
session_letter = 'A'  # 使用session A进行分析
data_root = '../data'

# 存储每个脑区的分类结果
results = {}

for region in target_brain_regions:
    print(f"\n======= 处理脑区: {region} =======")
    
    # 1. 读取元数据
    tgt_dict, tgt_folder = read_Decoding_csv(data_path=data_root, 
                                            region_type=region, 
                                            session_letter=session_letter)
    
    # 2. 加载神经活动数据
    neural_file_path = os.path.join(data_root, tgt_folder)
    for file_name in os.listdir(neural_file_path):
        if file_name.endswith('.npz'):
            ts, dff, all_roi_masks, cids, metadata = load_neural_npz(
                os.path.join(neural_file_path, file_name))
            break  # 假设每个文件夹只有一个npz文件
    
    # 3. 提取刺激时间段和创建标签
    stimulus_data = []
    stimulus_labels = []
    stimulus_types = []
    
    # 定义刺激类型映射
    stim_type_mapping = {
        'drifting_gratings': 0,
        'static_gratings': 1,
        'natural_scenes': 2,
        'natural_movie': 3  # 包括natural_movie_one和three
    }
    
    # 处理每种刺激类型
    for stim_key, stim_df in tgt_dict.items():
        if 'drifting_gratings' in stim_key:
            stim_type = 'drifting_gratings'
        elif 'static_gratings' in stim_key:
            stim_type = 'static_gratings'
        elif 'natural_scenes' in stim_key:
            stim_type = 'natural_scenes'
        elif 'natural_movie' in stim_key:  # 包括one和three
            stim_type = 'natural_movie'
        else:
            continue  # 跳过其他刺激类型
        
        # 获取刺激开始和结束时间
        for _, row in stim_df.iterrows():
            start_time = row['start']
            end_time = row['end']
            
            # 找到对应时间段的神经活动索引
            time_indices = np.where((ts >= start_time) & (ts <= end_time))[0]
            
            if len(time_indices) > 0:
                # 计算该时间段内所有神经元的平均活动
                neural_activity = np.mean(dff[:, time_indices], axis=1)
                stimulus_data.append(neural_activity)
                stimulus_labels.append(stim_type_mapping[stim_type])
                stimulus_types.append(stim_type)
    
    # 4. 准备分类数据
    X = np.array(stimulus_data)
    y = np.array(stimulus_labels)
    
    # 检查是否有足够样本
    if len(np.unique(y)) < 2:
        print(f"警告: 脑区 {region} 只有 {len(np.unique(y))} 类刺激，无法分类")
        results[region] = {'accuracy': None, 'classes': np.unique(y)}
        continue
    
    # 5. 数据预处理
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 6. 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.3, random_state=42, stratify=y)
    
    # 7. 训练分类器
    classifier = LogisticRegression(max_iter=1000, multi_class='ovr')
    classifier.fit(X_train, y_train)
    
    # 8. 评估性能
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # 9. 保存结果
    results[region] = {
        'accuracy': accuracy,
        'num_samples': len(y),
        'classes': np.unique(y),
        'class_names': [k for k, v in stim_type_mapping.items() if v in np.unique(y)]
    }
    print(f"脑区 {region} 分类准确率: {accuracy:.4f}")
    print(f"使用的刺激类型: {results[region]['class_names']}")

# 10. 比较三个脑区的性能
print("\n======= 分类结果比较 =======")
for region, res in results.items():
    if res['accuracy'] is not None:
        print(f"{region}脑区: 准确率={res['accuracy']:.4f}, 样本数={res['num_samples']}, 刺激类型={res['class_names']}")
    else:
        print(f"{region}脑区: 没有足够的刺激类型进行分类")


======= 处理脑区: VISp =======
we have keys in data_dict, including:
stimulus_metadata_drifting_gratings.csv
stimulus_metadata_natural_movie_one.csv
stimulus_metadata_natural_movie_three.csv
stimulus_metadata_spontaneous.csv
check subtitles, should be ['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
警告: 脑区 VISp 只有 1 类刺激，无法分类

======= 分类结果比较 =======
VISp脑区: 没有足够的刺激类型进行分类
